# **Tech Challenge**  👨🏻‍💻

**Problema:**

*   Você é um(a) profissional encarregado(a) de desenvolver um modelo preditivo de regressão para prever o valor dos custos médicos individuais cobrados pelo seguro de saúde.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# **Exploração de dados** 🕵🏻



*   Carregue a base de dados e explore suas características;
*   Analise estatísticas descritivas e visualize distribuições relevantes.

In [ ]:
df = pd.read_csv('data/healthcare_data_100k.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# Total de linhas nulas para cada coluna
# gênero e idade possuem linhas
df.isnull().sum()

In [ ]:
# Estatísticas descritivas
print(df.describe(include='all'))

In [ ]:
# Plotar histogramas para colunas numéricas
numerical_cols = df.select_dtypes(include='number').columns

plt.figure(figsize=(15, 12))
for i, col in enumerate(numerical_cols[:6]):  # limitar a 6 para melhor visualização
    plt.subplot(3, 2, i + 1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribuição de {col}')
plt.tight_layout()
plt.show()

'''
Na distribuição idade é possível notar:
    * Uma concentração entre 50 e 70 anos
    * Presença de valores muito altos acima de 800, distorcendo a escala
'''

In [ ]:
# Criar o gráfico de boxplot
plt.boxplot(df['renda_mensal'])
plt.title('renda_mensal')
plt.ylabel('Valores')
plt.show()

In [ ]:
plt.boxplot(df['imc'])
plt.title('imc')
plt.ylabel('Valores')
plt.show()

In [ ]:
plt.boxplot(df['filhos'])
plt.title('filhos')
plt.ylabel('Valores')
plt.show()

In [ ]:
print(df['renda_mensal'].max())
print(df['renda_mensal'].min())
print(df['renda_mensal'].mean())

# **Pré-processamento de dados** 🛠️
*   Realize a limpeza dos dados, tratando valores ausentes;
*   Converta variáveis categóricas em formatos adequados para modelagem.

Atributo idade e gênero possuem linhas vazias

In [ ]:
df['idade_ausente'] = df['idade'].isnull() | (df['idade'] < 1) | (df['idade'] > 105)

In [ ]:
df['idade_ausente'] = df['idade_ausente'].astype(int)

In [ ]:
# Preencher idade inválida ou ausente com a mediana das idades válidas
mediana_idade_valida = df.loc[(df['idade'] >= 1) & (df['idade'] <= 105), 'idade'].median()

In [ ]:
df['idade'] = df['idade'].apply(
    lambda x: mediana_idade_valida if pd.isnull(x) or x < 1 or x > 105 else x
)

# Mostrar as novas estatísticas
idade_minima_corrigida = df['idade'].min()
idade_maxima_corrigida = df['idade'].max()
mediana_idade_valida, idade_minima_corrigida, idade_maxima_corrigida

In [ ]:
# Gerar histograma da coluna 'idade'
plt.figure(figsize=(10, 6))
plt.hist(df['idade'].dropna(), bins=50, edgecolor='black')
plt.title('Histograma da Idade')
plt.xlabel('Idade')
plt.ylabel('Frequência')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# No gráfico podemos ver que muitos registros possuem a idade 99

contagem_idade_99 = (df['idade'] == 99).sum()
contagem_idade_999 = (df['idade'] == 999).sum()
contagem_idade_nulas = (df['idade'].isnull()).sum()
print(f"Quantidade de registros com idade 99: {contagem_idade_99}")
print(f"Quantidade de registros com idade 999: {contagem_idade_999}")
print(f"Quantidade de registros nulos: {contagem_idade_nulas}")


In [ ]:
# Solução 1 - Substituir por mediana.

# Calcular a mediana das idades válidas (excluindo nulos e o valor 99, que é considerado placeholder)
#mediana_idade_valida = df.loc[(df['idade'] != 99) & (~df['idade'].isnull()), 'idade'].median()

#print(f"Idade mediana: {mediana_idade_valida}")

# Substituir 99, 999 e NaN por 46.0
#df['idade'] = df['idade'].apply(lambda x: 46.0 if pd.isnull(x) or x == 999 or x == 99 else x)

In [ ]:
print(df['idade'].unique())

In [ ]:
media_idade = df['idade'].mean()
print(f"Média de idade: {media_idade:.2f} anos")

In [ ]:
# Distribuição da coluna 'gênero'
print(df['gênero'].value_counts(dropna=False))

In [ ]:
# Substituir valores nulos em 'gênero' pela moda (valor mais frequente)
#moda_genero = df['gênero'].mode()[0]
#print(moda_genero)

# Substituir nulos e 'desconhecido' por essa moda
#df['gênero'] = df['gênero'].replace(to_replace=[None, 'desconhecido'], value=moda_genero)
#df['gênero'] = df['gênero'].fillna(moda_genero)

# Verificar se ainda existem valores faltantes ou desconhecidos
#print(df['gênero'].value_counts(dropna=False))

In [ ]:
df['gênero'] = df['gênero'].replace([None, 'desconhecido'], 'não informado')
df['gênero'] = df['gênero'].fillna('não informado')

In [ ]:
# ========================
# GRÁFICOS DE BARRAS PARA VARIÁVEIS CATEGÓRICAS
# ========================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

sns.countplot(x='gênero', data=df, ax=axes[0, 0])
axes[0, 0].set_title('Contagem por Gênero')

sns.countplot(x='fumante', data=df, ax=axes[0, 1])
axes[0, 1].set_title('Contagem de Fumantes')

sns.countplot(x='região', data=df, ax=axes[0, 2])
axes[0, 2].set_title('Contagem por Região')

sns.countplot(x='tipo_plano', data=df, ax=axes[1, 0])
axes[1, 0].set_title('Tipo de Plano')

sns.countplot(x='uso_medicamento', data=df, ax=axes[1, 1])
axes[1, 1].set_title('Uso de Medicamento')

sns.countplot(x='doencas_cronicas', data=df, ax=axes[1, 2])
axes[1, 2].set_title('Doenças Crônicas')

plt.tight_layout()
plt.show()

In [ ]:
#print(df['encargos'].describe())
df['encargos'] = df['encargos'] / 1000
#print(df['encargos'].describe())

In [ ]:
print(df.dtypes)

In [ ]:
# Solução 1 - Categorizar utilizando get_dummies.

# Converta variáveis categóricas em formatos adequados para modelagem.


# df_encoded = pd.get_dummies(df, columns=['gênero', 'fumante', 'região',
#                                          'tipo_plano', 'uso_medicamento',
#                                          'doencas_cronicas'], drop_first=True)


# # Normalização de Colunas Numéricas
# scaler = StandardScaler()
# colunas_numericas = ['idade', 'imc', 'filhos', 'renda_mensal']
# df_encoded[colunas_numericas] = scaler.fit_transform(df_encoded[colunas_numericas])

# # Normalização de Colunas Booleanas
# df_encoded = df_encoded.astype({col: int for col in df_encoded.select_dtypes('bool').columns})

# print(df_encoded.dtypes)



In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Ajustar e transformar os rótulos
df['gênero'] = label_encoder.fit_transform(df['gênero'])
df['fumante'] = label_encoder.fit_transform(df['fumante'])
df['região'] = label_encoder.fit_transform(df['região'])
df['tipo_plano'] = label_encoder.fit_transform(df['tipo_plano'])
df['uso_medicamento'] = label_encoder.fit_transform(df['uso_medicamento'])
df['doencas_cronicas'] = label_encoder.fit_transform(df['doencas_cronicas'])

# **Feature Engineering** ✨

In [ ]:
df['tipo_plano_renda'] = df['tipo_plano'] * df['renda_mensal']
df['filhos_renda'] = df['filhos'] / (df['renda_mensal'] + 1)

df['idade_imc'] = df['idade'] * df['imc']
df['idade_filhos'] = df['idade'] * df['filhos']

#df['tem_filhos'] = (df['filhos'] > 0).astype(int)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe_transform = ohe.fit_transform(df[['tipo_plano']])

ohe.get_feature_names_out()

In [ ]:
ohe_transform.toarray()

In [ ]:
df_ohe = pd.DataFrame(ohe_transform.toarray())
df_ohe.columns = ohe.get_feature_names_out()
df_ohe.head()

In [ ]:
# Concatenar os dataframes
df = pd.concat([df, df_ohe], axis=1)
df.head()

In [ ]:
# Solução 2 -  Utilizando OrdinalEncoder o modelo não respondeu tão bem quanto com OneHotEncoder

#from sklearn.preprocessing import OrdinalEncoder

#oe = OrdinalEncoder()
#oe_transform_regiao = oe.fit_transform(df[['região']])
#oe_transform_plano = oe.fit_transform(df[['tipo_plano']])

#df['num_tipo_plano'] = oe_transform_plano

In [ ]:
def categorizar_idade(idade):
    if idade < 30:
        return 'jovem'
    elif idade < 45:
        return 'adulto'
    elif idade < 60:
        return 'meia_idade'
    elif idade < 75:
        return 'idoso'
    else:
        return 'muito_idoso'

df['faixa_etaria'] = df['idade'].apply(categorizar_idade)


In [ ]:
ohe_transform_faixa_etaria = ohe.fit_transform(df[['faixa_etaria']])

ohe.get_feature_names_out()

In [ ]:
ohe_transform_faixa_etaria.toarray()

In [ ]:
df_ohe = pd.DataFrame(ohe_transform_faixa_etaria.toarray())
df_ohe.columns = ohe.get_feature_names_out()
df_ohe.head()

In [ ]:
# Concatenar os dataframes
df = pd.concat([df, df_ohe], axis=1)
df.head()

In [ ]:
df = df.drop(columns=['faixa_etaria', 'idade', 'tipo_plano'])
df.head()

In [ ]:
# Selecionar variáveis numéricas para o heatmap
corr_df = df[['imc', 'filhos',
              'renda_mensal', 'encargos',
              'idade_imc', 'filhos_renda', 
              'idade_filhos', 'gênero']]

# Calcular matriz de correlação
correlation_matrix = corr_df.corr()

# Plotar heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Mapa de Correlação entre Variáveis Numéricas')
plt.tight_layout()
plt.show()

# **Criação das variáveis derivadas** ⚙️

In [ ]:
# Calcular Q1, Q3 e IQR
Q1 = df['renda_mensal'].quantile(0.25)
Q3 = df['renda_mensal'].quantile(0.75)
IQR = Q3 - Q1

# Limites superior e inferior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Aplicar truncamento dos valores fora do intervalo
df['renda_mensal'] = df['renda_mensal'].clip(lower=limite_inferior, upper=limite_superior)



plt.boxplot(df['renda_mensal'])
plt.title('renda_mensal (após truncamento dos outliers)')
plt.ylabel('Valores')
plt.show()

In [ ]:
df['obeso'] = (df['imc'] >= 30).astype(int)

df.head()

In [ ]:
from sklearn.linear_model import LinearRegression

X = df.drop(columns=['encargos'])
y = df['encargos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar modelo de regressão linear
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

# Métricas
rmse_antes = np.sqrt(mean_squared_error(y_test, y_pred))
r2_antes = r2_score(y_test, y_pred)

print(f"RMSE: {rmse_antes:.2f}")
print(f"R²: {r2_antes:.2f}")

📈 Resumo da Regressão Linear (OLS)

| Métrica                | Valor           | Interpretação                                            |
| ---------------------- | --------------- | -------------------------------------------------------- |
| **R-squared**          | 0.010           | O modelo explica apenas 1% da variabilidade dos encargos |
| **Adj. R-squared**     | 0.008           | Valor ajustado ainda menor, indicando pouca eficácia     |
| **F-statistic**        | 7.017           | Teste global do modelo                                   |
| **Prob (F-statistic)** | 1.24e-14        | Modelo como um todo é estatisticamente significativo     |
| **Log-Likelihood**     | -109410         | Critério para comparação entre modelos                   |
| **AIC / BIC**          | 218900 / 219000 | Quanto menores, melhor (útil para comparar modelos)      |
| **Nº de observações**  | 10.000          | Dataset robusto                                          |
| **Df Model**           | 14              | Número de variáveis independentes                        |

✅ Conclusão do Modelo OLS

Apesar da significância estatística global, o baixo R² mostra que o modelo linear simples não tem poder preditivo adequado. Ele é útil para interpretação, mas não suficiente para previsão prática.

# **Treinamento Random Forest** 🌲🌳🌿

In [ ]:
# 1. Separar features e variável alvo

#X = df.drop(columns='encargos')

X = df.drop('encargos', axis=1)
y = df['encargos']

In [ ]:
# 2. Dividir em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
# # 6. Treinar modelo Random Forest
#modelo_rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
modelo_rf = RandomForestRegressor(n_estimators=400, random_state=0, n_jobs=-1, min_samples_leaf=1)
modelo_rf.fit(X_train, y_train)

In [ ]:
# # 7. Fazer predições
y_pred = modelo_rf.predict(X_test)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# # 8. Avaliar desempenho

#Baseline

baseline_pred = np.full_like(y_test, y_test.mean())

rmse_baseline = mean_squared_error(y_test, baseline_pred)
mae_baseline = mean_absolute_error(y_test, baseline_pred)

#Modelo

rmse_modelo = mean_squared_error(y_test, y_pred)
mae_modelo = mean_absolute_error(y_test, y_pred)

# 3. Porcentagem de melhora
melhora_rmse = (rmse_baseline - rmse_modelo) / rmse_baseline * 100
melhora_mae = (mae_baseline - mae_modelo) / mae_baseline * 100

r2 = r2_score(y_test, y_pred)

#print(f"RMSE (Erro Quadrático Médio): {rmse_modelo:.2f}")
#print(f"R² (Coeficiente de Determinação): {r2:.2f}")

print(f"📈 RMSE: {rmse_modelo:.2f}")
print(f"📉 MAE: {mae_modelo:.2f}")
print(f"🎯 R²: {r2:.2f}%\n")

print(f"📊 RMSE Baseline: {rmse_baseline:.2f}")
print(f"📈 RMSE Modelo  : {rmse_modelo:.2f}")
print(f"✅ Redução RMSE : {melhora_rmse:.2f}%\n")

print(f"📊 MAE Baseline : {mae_baseline:.2f}")
print(f"📉 MAE Modelo   : {mae_modelo:.2f}")
print(f"✅ Redução MAE  : {melhora_mae:.2f}%")

📊 Interpretação dos Resultados:

| Métrica  | Valor | Interpretação                                                                                   |
| -------- | ----- | ----------------------------------------------------------------------------------------------- |
| 📊**Baseline (média)** | 189.62 | Modelo que sempre prevê a média dos encargos.                              |
| 📈 **RMSE** | 39.25 | Em média, o modelo erra os encargos em ±62.83 unidades monetárias.Erro médio quadrático — sensível a outliers. Um valor muito bom considerando a base original com RMSE baseline de 189.62. |
| 📉 **MAE** | 4.19 | Erro médio absoluto — mostra que, em média, o modelo erra apenas 4.19 unidades, o que é excelente. |
| 🎯 **R²**   | 0.79  | O modelo explica **79% da variação** dos encargos com as variáveis usadas. |


In [ ]:
# Real vs Previsto

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Encargos Reais vs. Previstos (Modelo Leve)')
plt.xlabel('Encargos Reais')
plt.ylabel('Encargos Previstos')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Importâncias das features
importancias = pd.Series(modelo_rf.feature_importances_, index=X.columns)
importancias_top = importancias.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=importancias_top.values, y=importancias_top.index)
plt.title('Top 15 Features Mais Importantes')
plt.xlabel('Importância')
plt.ylabel('Variáveis')
plt.tight_layout()
plt.show()

In [ ]:

# 3. Gráfico: Resíduos (Erros)
residuos = y_test - y_pred
plt.figure(figsize=(8,6))
sns.histplot(residuos, bins=40, kde=True)
plt.axvline(0, color='r', linestyle='--')
plt.xlabel('Erro (y_real - y_previsto)')
plt.title('Distribuição dos Resíduos')
plt.grid(True)
plt.tight_layout()
plt.show()

✅ 1. Forma Simétrica e Centragem em 0
* A distribuição está bem centrada no zero, com uma leve cauda à direita.

* Isso indica que o modelo não está sistematicamente errando para mais ou para menos, o que é ótimo.

✅ 2. Pico alto no centro
* A maioria dos resíduos está bem próxima de 0, sugerindo que as previsões estão muito próximas dos valores reais para grande parte dos dados.

⚠️ 3. Caudas levemente assimétricas
* Pequena assimetria para a direita (resíduos positivos mais longos) indica que o modelo subestima ligeiramente os encargos mais altos.

* Isso pode acontecer em problemas de regressão com outliers positivos ou valores raros muito altos.

In [ ]:
# 5. Gráfico comparativo
labels = ['RMSE', 'MAE']
baseline_vals = [rmse_baseline, mae_baseline]
modelo_vals = [rmse_modelo, mae_modelo]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(7,5))
ax.bar(x - width/2, baseline_vals, width, label='Baseline', color='gray')
ax.bar(x + width/2, modelo_vals, width, label='Modelo', color='green')

ax.set_ylabel('Erro')
ax.set_title('Comparação de Erros: Baseline vs. Modelo')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
#import joblib
#joblib.dump(modelo_rf, 'modelo_encargos_rf.pkl')